In [1]:
%tensorflow_version 1.15
from google.colab import drive
drive.mount('/content/gdrive')

`%tensorflow_version` only switches the major version: 1.x or 2.x.
You set: `1.15`. This will be interpreted as: `1.x`.


TensorFlow 1.x selected.
Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/gdrive


In [0]:
import tensorflow as tf
import matplotlib.pyplot as plt
import numpy as np
import logging
import os
from PIL import Image
import numpy as np
import random
import time
import json
import math

In [0]:
#walks through the cfg file to get the information about the architecture and hyperparameters
def parsecfg(filedir):
    file = open(filedir, 'r')
    lines = file.read().split('\n')                        
    lines = [x for x in lines if len(x) > 0]               
    lines = [x for x in lines if x[0] != '#']              
    lines = [x.rstrip().lstrip() for x in lines]           

    block = {}
    blocks = []

    for line in lines:
      if line[0] == "[":               
        if len(block) != 0:         
          blocks.append(block)     
          block = {}               
        block["type"] = line[1:-1].rstrip()    
      else:
        key,value = line.split("=")
        block[key.rstrip()] = value.lstrip()
    blocks.append(block)
    return blocks

In [4]:
blocks=parsecfg("/content/gdrive/My Drive/CS559/HWNet_6_v14.cfg")
#creates the hyperparameters dictionary
hyperparameters = {}
for key in blocks[0]:
  val = blocks[0][key]
  hyperparameters[key]=val
print(hyperparameters)

{'type': 'net', 'img_size': '80', 'train_color': 'rgb', 'batch_normalization': '1', 'dropout': '1', 'beta1': '0.009', 'beta': '0.0001', 'beta2': '0.9991', 'epsilon': '1e-08', 'learning_rate': '0.0001', 'batch_size': '128', 'epoch': '500', 'early': '0', 'loss': 'mae', 'init': 'xavier'}


In [0]:
def data_loader(file_dir, num_data, size, channel_type):
    print("Started loading data")
    images = []
    labels = []
    count  = 0
    
    all_files = os.listdir(file_dir)
    #since the dataset has given us in a sorted order we need to shuffle the dataset
    #instead of using some fancy method we just go straightforward and randomly get the images
    r = list(range(num_data))
    random.shuffle(r)
    for i in r:
        if channel_type =="greyscale":
          channel= 1
          im = Image.open(file_dir+all_files[i],"r").convert('L') #getting the image, converting it to the greyscale
        else:
          channel=3
          im = Image.open(file_dir + all_files[i],"r")            #getting the image
        label = all_files[i].split(".jpg")[0].split("_")[0]       #getting the label from name
        temp_data = np.asarray(im.getdata())                      #converting the image -> pixels
        pix_val = np.resize(temp_data,(size, size, channel))      #the data already in expected size but just in case
        pix_val = np.true_divide(pix_val, 255)                    #normalization of pixels
        images = np.append(images, pix_val)                       #putting all pixels to one array, will be reshaped later
        labels = np.append(labels, label)                         #putting all labels to one array, will be reshaped later 
        count = count + 1                                         #counting the number of samples

    images = np.reshape(images, (count, size,size,channel))       #pixels reshaped
    images = images.astype('float32')
    labels = np.reshape(labels, (count, 1))                       #labels reshaped
    labels = labels.astype('float32')

    print("shape for images: " + str(images.shape))               #just to make sure everything is retrieved as we wanted
    print("shape for labels: " + str(labels.shape))             

    return images, labels

In [6]:
print("Getting the training data")
#If the greyscale selected the number of channels will be equal to 1
#if RGB selected, the number of channels will be equal to 3
#These values also passed to the dataLoader method to retrieve data accordingly
if hyperparameters['train_color']=='greyscale':
  num_channels= 1
elif hyperparameters['train_color']=='rgb':
  num_channels = 3

#According to the file directory the images are retrieved and set to the values.
file_dir_train = "/content/gdrive/My Drive/CS559/SCUT_FBP5500_downsampled/training/"
images, labels = data_loader(file_dir_train,3550, int(hyperparameters['img_size']) , hyperparameters['train_color']) #3550
print(hyperparameters['train_color'])

print("Getting the validation data")
file_dir_val = "/content/gdrive/My Drive/CS559/SCUT_FBP5500_downsampled/test/"
images_val, labels_val = data_loader(file_dir_val, 892, int(hyperparameters['img_size']), hyperparameters['train_color']) #892

print("Getting the testing data")
file_dir_test = "/content/gdrive/My Drive/CS559/SCUT_FBP5500_downsampled/validation/"
images_test, labels_test = data_loader(file_dir_test, 893, int(hyperparameters['img_size']), hyperparameters['train_color']) #893

Getting the training data
Started loading data
shape for images: (3550, 80, 80, 3)
shape for labels: (3550, 1)
rgb
Getting the validation data
Started loading data
shape for images: (892, 80, 80, 3)
shape for labels: (892, 1)
Getting the testing data
Started loading data
shape for images: (893, 80, 80, 3)
shape for labels: (893, 1)


In [0]:
#Creating the conv layer for the network
#if the net part of the config file has 1 for batch normalization we activate the batch normalization here
def conv2D(x, W, b, stride = 1, pad = 'SAME'):
    x = tf.nn.conv2d(x, W, stride, pad)
    x = tf.nn.bias_add(x, b)
    #If you want to measure the effect of the batch normalization change it to 0 from configuration file
    if hyperparameters['batch_normalization'] == 1:
      mean, var = tf.nn.moments(x, [0,1,2], name='moments')
      x = tf.nn.batch_normalization(x, mean, var, variance_epsilon = 1e-3, name=None)
    #ReLU activation function was selected as the major activation
    x = tf.nn.relu(x)
    return  x

#Method for the max pooling
#If stride is 1 then the shape of X wont change
#If stride is 2 then the shape of X will reduced
#Since we use placeholders while implementing the code the parameters that will initialize before the forward pass will not 
#effect from the change in size. 
def maxpool2d(x, k):
    #change its value to 1 from config file in order to observe the effect of it. 
    if hyperparameters['dropout'] == 1:
      x = tf.nn.dropout( x, 0.5)
    ksize = [1,k,k,1]
    strides= [1,k,k,1]
    return tf.nn.max_pool2d(x, ksize, strides,padding='SAME')

#we need placeholders to pass data in when the session will be run.
#Just a default method we needed because we are using Tensorflow version 1.15
#Things change in the new versions of TF
def create_placeholders(n_s, n_xc, n_y):
    X = tf.placeholder(tf.float32, [None, n_s, n_s, n_xc], name="X")
    Y = tf.placeholder(tf.float32, [None, n_y], name="Y")    
    return X, Y

#In order to observe the effect of different loss functions change the values of loss from the config file 
def getLoss(loss, Y, pred):
    #Loss for Mean Squared Error
    if loss == "mse":
        return tf.compat.v1.losses.mean_squared_error(labels=Y, predictions=pred)
    #Loss for Mean Average Error
    elif loss == "mae":
        return tf.reduce_mean(tf.abs(tf.subtract(Y, pred)))
    #Loss for Log-cosh Error
    elif loss == "logcosh":
        return tf.math.reduce_sum(tf.math.log(tf.cosh(pred - Y)))

In [8]:
#Method to initialize the parameters which will be get ready for the forward pass
def initialize_parameters(blocks):
  parameters_w = {}                          #the placeholder parameters for weights will be stored as dictionary
  parameters_b = {}                          #the placeholder parameters for biases will be stored as dictionary
  stride_counter=0                           #in order to decide the input shape for the first fully connected layer
  num_filter = []                            #since as input the previous layers size used we had to store all the size of the values
  img_size = hyperparameters['img_size']     #the size of the image from config file
  init= hyperparameters['init']              #the initializer type from config file
  initializer =None                          #the value of it will be given in teh following if-else bllock
  fc_flag=False                              #flag to trace the first fully connected layer
  #If-else block to decide initialization method
  #If you want to observe the effect of initialization on the results change values from config file.
  if init =='xavier':
    print(init + " initialized")
    initializer= tf.contrib.layers.xavier_initializer()
  elif init == 'gauss':
    print(init + " initialized")
    initializer = tf.compat.v1.random_uniform_initializer()

  with tf.compat.v1.variable_scope("init",reuse=tf.compat.v1.AUTO_REUSE): #TFv15 requirement
    #walks through each block value to initialize parameters
    for i in range(len(blocks)):
      module_type = blocks[i]['type']                   #gets the type of the layer
      if i ==0:
        num_filter=np.append(num_filter, 0)             #since the first part of the config file consist of the values of hyperparameters
      else:
        w_name = 'w'+str(i)                             #created names of the weights for the dictionary
        b_name = 'b'+str(i)                             #created names of the biases  for the dictionary
        stride = int(blocks[i]['stride'])               #gets the valur of stride
        if stride ==2:                                  #counts the number of stride =2 because we need these to create the first FC layer
          stride_counter=stride_counter+1           
        kernel = int(blocks[i]['kernel'])               #gets the value of the size of the kernel
        num_filters = int(blocks[i]['filter'])          #gets the value of the size of the filters
        num_filter = np.append(num_filter, num_filters) #pushes value of the size of the filters to the array
        #If the layer type is convolutional the parameters for the conv layers created
        if module_type == 'conv':                       
          #Since the first convolutional layer takes the image tensor as input we need to distinguish the first layer from others
          if i ==1:
            parameters_w[i] = tf.compat.v1.get_variable(w_name, shape = [kernel, kernel, num_channels, num_filters]   , initializer=initializer)
            parameters_b[i] = tf.compat.v1.get_variable(b_name ,shape = (num_filters) , initializer = initializer)
          else:
            parameters_w[i] = tf.compat.v1.get_variable(w_name, shape = [kernel, kernel, num_filter[i-1], num_filters]   , initializer = initializer)
            parameters_b[i] = tf.compat.v1.get_variable(b_name ,shape = (num_filters) , initializer = initializer)
        #If the layer type is fully connected, the parameters for the densed layers are created
        elif module_type == 'fully':
          if fc_flag is False:
            dense =int(img_size)  
            for j in range(stride_counter):
              dense = dense / 2
              dense = int(math.ceil(dense))
              print(dense)
            parameters_w[i] = tf.compat.v1.get_variable(w_name , shape = [(dense*dense*num_filters), num_filters], initializer = initializer)
            parameters_b[i] = tf.compat.v1.get_variable(b_name, shape = (num_filters), initializer = initializer)
            fc_flag=True
          else:
            parameters_w[i] = tf.compat.v1.get_variable(w_name , shape = [num_filter[i-1], num_filters], initializer = initializer)
            parameters_b[i] = tf.compat.v1.get_variable(b_name, shape = (num_filters), initializer = initializer)
        #If the layer type is regression, the parameters for the last layers are created
        elif module_type == 'regression':
          parameters_w[i] = tf.compat.v1.get_variable(w_name , shape = [num_filter[i-1], 1] , initializer = initializer)
          parameters_b[i] = tf.compat.v1.get_variable(b_name , shape = (1), initializer = initializer)
    return parameters_w, parameters_b 

xavier initialized
The TensorFlow contrib module will not be included in TensorFlow 2.0.
For more information, please see:
  * https://github.com/tensorflow/community/blob/master/rfcs/20180907-contrib-sunset.md
  * https://github.com/tensorflow/addons
  * https://github.com/tensorflow/io (for I/O related ops)
If you depend on functionality not listed there, please file an issue.

40
20
10


{1: <tf.Variable 'init/w1:0' shape=(3, 3, 3, 32) dtype=float32_ref>,
 2: <tf.Variable 'init/w2:0' shape=(3, 3, 32, 32) dtype=float32_ref>,
 3: <tf.Variable 'init/w3:0' shape=(3, 3, 32, 32) dtype=float32_ref>,
 4: <tf.Variable 'init/w4:0' shape=(3, 3, 32, 64) dtype=float32_ref>,
 5: <tf.Variable 'init/w5:0' shape=(3, 3, 64, 64) dtype=float32_ref>,
 6: <tf.Variable 'init/w6:0' shape=(3, 3, 64, 64) dtype=float32_ref>,
 7: <tf.Variable 'init/w7:0' shape=(3, 3, 64, 64) dtype=float32_ref>,
 8: <tf.Variable 'init/w8:0' shape=(3, 3, 64, 128) dtype=float32_ref>,
 9: <tf.Variable 'init/w9:0' shape=(3, 3, 128, 128) dtype=float32_ref>,
 10: <tf.Variable 'init/w10:0' shape=(3, 3, 128, 128) dtype=float32_ref>,
 11: <tf.Variable 'init/w11:0' shape=(12800, 128) dtype=float32_ref>,
 12: <tf.Variable 'init/w12:0' shape=(128, 128) dtype=float32_ref>,
 13: <tf.Variable 'init/w13:0' shape=(128, 64) dtype=float32_ref>,
 14: <tf.Variable 'init/w14:0' shape=(64, 64) dtype=float32_ref>,
 15: <tf.Variable 'init

In [0]:
#Forward propogation 
#X : tensor for input images
#p_w: dictionary which stores the weights(initialized, with placeholders)
#p_b: dictionary which stores the biases (initialized, with placeholders)
def forward_propagation(X, p_w, p_b):    
    layers = {}
    fc_flag=False                                               #in forward propogation we needed flag for FC as well, because we only need to reshape the layer once
    for i, module in enumerate(blocks):                         #walks through the blocks
      module_type = (module['type'])
      if module_type == 'conv':
        stride = int(blocks[i]['stride'])
        layer_name = str(i)
        if bool(layers) == False:
          layers[layer_name] = conv2D(X, p_w[i] , p_b[i])         #pass the value of input tensor
          layers[layer_name] = maxpool2d(layers[str(i)], k=stride)#maxs pools the convolutional layer
          print(str(layers[layer_name].shape))                    #prints the shape of the layer to double check the architecture
        else:
          layers[layer_name] = conv2D(layers[str(i-1)], p_w[i] , p_b[i])
          layers[layer_name] = maxpool2d(layers[str(i)], k=stride)
          print(str(layers[layer_name].shape))
      elif module_type == 'fully':
        if fc_flag is False:
          layer_name=str(i)
          layers[layer_name] = tf.reshape(layers[str(i-1)], [-1, parameters_w[i].get_shape().as_list()[0]]) #reshapes the result of the previous convolutional layer
          layers[layer_name] = tf.add(tf.matmul(layers[layer_name], parameters_w[i]), parameters_b[i])
          if hyperparameters['batch_normalization'] == 1:
              mean, var = tf.nn.moments(layers[layer_name], [0], name='moments')
              layers[layer_name] = tf.nn.batch_normalization(layers[layer_name], mean, var, variance_epsilon = 1e-3, name=None) #applies batch normalization to FC as well
          layers[layer_name] = tf.nn.relu(layers[str(i)])
          if hyperparameters['dropout'] == 1:
            layers[layer_name] = tf.nn.dropout(layers[layer_name], 0.5)
          print(layers[layer_name].shape)
        else:
          layers[layer_name] = tf.add(tf.matmul(layers[str(i-1)], parameters_w[i]), parameters_b[i])
          if hyperparameters['batch_normalization'] == 1:
            mean, var = tf.nn.moments(layers[layer_name], [0], name='moments')
            layers[layer_name] = tf.nn.batch_normalization(layers[layer_name], mean, var, variance_epsilon = 1e-3, name=None)
          layers[layer_name] = tf.nn.relu(layers[str(i)])
          if hyperparameters['dropout'] == 1:
            layers[layer_name] = tf.nn.dropout(layers[layer_name], 0.5)             #applies dropout for the fully connected layer
          print(layers[layer_name].shape)
          print(deneme)
      elif module_type == 'regression':
        reg_layer = tf.add(tf.matmul(layers[str(i-1)], p_w[i]), p_b[i])
        print(reg_layer.shape)
        return reg_layer
  
    return "Error: There is no regression layer. Add regression layer to the cfg file"

In [10]:
X = tf.placeholder("float")
Y = tf.placeholder("float")

n_samples = images.shape[0]
parameters_w, parameters_b = initialize_parameters(blocks)
pred = forward_propagation(X, parameters_w, parameters_b)
loss = getLoss(hyperparameters['loss'], Y, pred)
regularizers = 0
for i in range(len(parameters_w)):
  regularizers = regularizers + tf.nn.l2_loss(parameters_w[i+1])
loss = loss + float(hyperparameters['beta']) * regularizers
with tf.compat.v1.variable_scope("opti",reuse=tf.compat.v1.AUTO_REUSE):
    optimizer = tf.compat.v1.train.AdamOptimizer(float(hyperparameters['learning_rate']), float(hyperparameters['beta1']), float(hyperparameters['beta2']), float(hyperparameters['epsilon'])).minimize(loss)
# Round predictions for MAE calculation
predEr = tf.math.round(pred)
error = tf.compat.v1.metrics.mean_absolute_error(Y, predictions=predEr)

xavier initialized
40
20
10
(?, ?, ?, 32)
(?, ?, ?, 32)
(?, ?, ?, 32)
(?, ?, ?, 64)
(?, ?, ?, 64)
(?, ?, ?, 64)
(?, ?, ?, 64)
(?, ?, ?, 128)
(?, ?, ?, 128)
(?, ?, ?, 128)
(?, 128)
(?, 128)
(?, 64)
(?, 64)
(?, 32)
(?, 32)
(?, 16)
(?, 1)


In [11]:
batch_size=int(hyperparameters['batch_size'])
epoch= int(hyperparameters['epoch'])
init = tf.compat.v1.global_variables_initializer()
prev_val_err = 0
val_errors = []
firstIter = 1
with tf.compat.v1.Session() as sess:
    print("Session started")
    trainLoss=[]
    valLoss=[]
    valAcc=[]
    sess.run(init)
    sess.run(tf.compat.v1.local_variables_initializer())
    val_X = images_val
    val_Y = labels_val
    for ithepoch in range(int(epoch)):
        tic = time.perf_counter()
        loss_train = None
        for batch in range(len(images)//batch_size):
          batch_x = images[batch*batch_size:min((batch+1)*batch_size,len(images))]    #splits the data to the bacthes for images
          batch_y = labels[batch*batch_size:min((batch+1)*batch_size,len(images))]    #splits the data to the bacthes for labels
          sess.run(optimizer, feed_dict={X: batch_x, Y: batch_y})                     #runs the optimizer which actually makes the training
          loss_train = sess.run(loss, feed_dict={X: batch_x, Y:batch_y})              #runs the loss which already calculated when we run the optimizer and gave us the result
        
        print("Epoch:", '%04d' % (ithepoch+1), "trainLoss=",loss_train)               #shows the result of each epoch
        
        loss_val = sess.run(loss, feed_dict={X:images_val, Y:labels_val})
        err_val = sess.run(error,feed_dict={X:images_val, Y:labels_val})
        '''if hyperparameters['early'] == 1:
          if firstIter == 1 and prev_val_err == 0:
            prev_val_err = err_val
            if firstIter != 1 and prev_val_err != 0:
                difference = err_val - prev_val_err
                val_errors.append(difference)
                prev_val_err = err_val
                if len(val_errors) == 5:
                    if np.amax(val_errors) > 0:
                        break;
                    val_errors.pop(0)
            firstIter == 0
            '''
        valLoss=np.append(valLoss, loss_val)
        valAcc=np.append(valAcc, err_val)
        print("            valLoss=",loss_val, "valAccuracy=",err_val[0])
        toc = time.perf_counter()
        print(f"\trun one epoch is in {toc - tic:0.4f} seconds")
        print()
    loss_test = sess.run(loss, feed_dict={X:images_test, Y:labels_test})
    err_test = sess.run(error,feed_dict={X:images_test, Y:labels_test})
    print()
    print("TEST RESULTS")
    print("Loss for Test Data: " + str(loss_test))
    print("Error for Test Data: " + str(err_test[0]))

Session started
Epoch: 0001 trainLoss= 1.696513
            valLoss= 1.6715876 valAccuracy= 0.0
	run one epoch is in 8.1760 seconds

Epoch: 0002 trainLoss= 1.701939
            valLoss= 1.671857 valAccuracy= 1.6199552
	run one epoch is in 1.2617 seconds

Epoch: 0003 trainLoss= 1.6933612
            valLoss= 1.6608795 valAccuracy= 1.5756726
	run one epoch is in 1.2624 seconds

Epoch: 0004 trainLoss= 1.6588397
            valLoss= 1.6251173 valAccuracy= 1.558296
	run one epoch is in 1.2567 seconds

Epoch: 0005 trainLoss= 1.6046062
            valLoss= 1.586374 valAccuracy= 1.5400785
	run one epoch is in 1.2580 seconds

Epoch: 0006 trainLoss= 1.5781631
            valLoss= 1.5900959 valAccuracy= 1.5468609
	run one epoch is in 1.2558 seconds

Epoch: 0007 trainLoss= 1.5389076
            valLoss= 1.5433893 valAccuracy= 1.5440956
	run one epoch is in 1.2534 seconds

Epoch: 0008 trainLoss= 1.5150763
            valLoss= 1.5115088 valAccuracy= 1.5358745
	run one epoch is in 1.2585 seconds

Epo